In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import string
from string import punctuation
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping 

In [21]:
import sklearn.linear_model

In [22]:
import sklearn.metrics

In [2]:
#pip install tensorflow


In [3]:
pwd

'C:\\Users\\Daniel\\OneDrive\\Documents\\MScData\\Semester2\\Adv Data Analysis'

In [24]:
df =pd.read_csv('TwitterDataSentiments.csv')

C:\Users\Daniel\AppData\Local\Temp\ipykernel_1712\2797416005.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df =pd.read_csv('TwitterDataSentiments.csv')


In [25]:
df

,Unnamed: 0,Date,Text,sentiments
0,0,06/04/2009 22:19,switchfoot httptwitpiccomyzl a s bummer you sh...,Netural
1,1,06/04/2009 22:19,upset cant update facebook texting it might cr...,Netural
2,2,06/04/2009 22:19,kenichan i dived many times ball managed save ...,Positive
3,3,06/04/2009 22:19,whole body feels itchy like fire,Positive
4,4,06/04/2009 22:19,nationwideclass no not behaving all im mad her...,Negative
...,...,...,...,...
2097145,2097145,NaN,opened mouth insert foot chewed really hard ho...,Negative
2097146,2097146,NaN,trying move head potteringabout mode backtowor...,Netural
2097147,2097147,NaN,sick twice already im still going school feel ...,Negative
2097148,2097148,NaN,kherrylobrien boo ill blue side gotta keep mag...,Netural


In [26]:
df = df.drop(['Unnamed: 0', ],axis=1)
df.dropna(inplace=True) 

In [58]:
df = df[['Text', 'sentiments']]
df.head() 


,Text,sentiments
0,switchfoot httptwitpiccomyzl a s bummer you sh...,Netural
1,upset cant update facebook texting it might cr...,Netural
2,kenichan i dived many times ball managed save ...,Positive
3,whole body feels itchy like fire,Positive
4,nationwideclass no not behaving all im mad her...,Negative


In [63]:
one_hot = pd.get_dummies(df["sentiments"])
df.drop(["sentiments"],axis=1, inplace=True)
df = pd.concat([df, one_hot], axis=1)
df.head() 


C:\Users\Daniel\AppData\Local\Temp\ipykernel_1712\3686950812.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(["sentiments"],axis=1, inplace=True)


,Text,Negative,Netural,Positive
0,switchfoot httptwitpiccomyzl a s bummer you sh...,0,1,0
1,upset cant update facebook texting it might cr...,0,1,0
2,kenichan i dived many times ball managed save ...,0,0,1
3,whole body feels itchy like fire,0,0,1
4,nationwideclass no not behaving all im mad her...,1,0,0


In [27]:
#sns.set_style('whitegrid')
#'sns.countplot(x='Text',data=df,
#palette='YlGnBu_r') 

In [64]:
df = df.head(10000)

In [65]:
X = df["Text"].values
y = df.drop(["Text"], axis=1).values
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [67]:
#Apply vectorization:
vect = CountVectorizer()
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test) 

In [68]:
#Apply frequency, inverse document frequency:
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)
X_train = X_train.toarray()
X_test = X_test.toarray()

In [69]:
#Build a Model with Deep Learning
#Add different layers to models:
model = Sequential()
model.add(Dense(units=12673, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units=4000, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units=500, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units=3, activation="softmax"))
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt,
metrics=["accuracy"])
early_stop = EarlyStopping(monitor="val_loss", mode="min", verbose=1,
patience=2)

In [70]:
#Fit the model:
model.fit(x=X_train,y=y_train,batch_size=256,epochs=100,validation_data=(X_test, y_test),verbose=1,callbacks=early_stop,
) 

Epoch 1/100
28/28 [==============================] - 115s 4s/step - loss: 0.8478 - accuracy: 0.5899 - val_loss: 0.6217 - val_accuracy: 0.7857
Epoch 2/100
28/28 [==============================] - 117s 4s/step - loss: 0.1420 - accuracy: 0.9561 - val_loss: 0.6312 - val_accuracy: 0.7783
Epoch 3/100
28/28 [==============================] - 123s 4s/step - loss: 0.0193 - accuracy: 0.9943 - val_loss: 0.8263 - val_accuracy: 0.8033
Epoch 3: early stopping
